In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_iris
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
        
import datetime

from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from collections import defaultdict

In [ ]:
start_time = datetime.datetime.now()
demographic_cols = ['fecha_dato',
 'ncodpers','ind_empleado','pais_residencia','sexo','age','fecha_alta','ind_nuevo','antiguedad','indrel','ult_fec_cli_1t',
 'indrel_1mes','tiprel_1mes','indresi','indext','conyuemp','canal_entrada','indfall',
 'tipodom','cod_prov','nomprov','ind_actividad_cliente','renta','segmento']

notuse = ["ult_fec_cli_1t","nomprov"]

product_col = [
 'ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']

train_cols = demographic_cols + product_col


bank_data = pd.read_csv("/kaggle/input/santander-pr/train.csv",dtype = {'ind_nuevo': 'float32' ,'indrel' : 'float32','ind_actividad_cliente': 'float32','ncodpers' : 'int32'})
df_test=pd.read_csv("/kaggle/input/santander-pr/test.csv",dtype = {'ind_nuevo': 'float32' ,'indrel' : 'float32','ind_actividad_cliente': 'float32','ncodpers' : 'int32'},usecols = demographic_cols)


# Divinding train data based on fecha_dato

In [ ]:
bank_data['fecha_dato'].unique()

In [ ]:
df_test['fecha_dato'].unique()

In [ ]:
#bank_data.loc[bank_data.fecha_dato('2015-04-28T00:00:00.000000000')]
#bank_data7 = bank_data.loc[bank_data['fecha_dato'] == '2015-08-28']
bank_data8 = bank_data.loc[bank_data['fecha_dato'] == '2015-02-28']
bank_data7 = bank_data.loc[bank_data['fecha_dato'] == '2015-03-28']
bank_data6 = bank_data.loc[bank_data['fecha_dato'] == '2015-04-28']
bank_data5 = bank_data.loc[bank_data['fecha_dato'] == '2015-05-28']
bank_data1 = bank_data.loc[bank_data['fecha_dato'] == '2016-01-28']
bank_data2 = bank_data.loc[bank_data['fecha_dato'] == '2016-02-28']
bank_data3 = bank_data.loc[bank_data['fecha_dato'] == '2016-03-28']
bank_data4 = bank_data.loc[bank_data['fecha_dato'] == '2016-04-28']
# bank_data7 = bank_data.loc[bank_data['fecha_dato'] == '2015-06-28']
# bank_data8 = bank_data.loc[bank_data['fecha_dato'] == '2015-07-28']
# bank_data9 = bank_data.loc[bank_data['fecha_dato'] == '2015-08-28']

Replacing the nan values in cod_prod feature with '0'

In [ ]:
df_test.loc[df_test.cod_prov.isnull(),"cod_prov"] = 0
# bank_data0.loc[bank_data0.cod_prov.isnull(),"cod_prov"] = 0
bank_data1.loc[bank_data1.cod_prov.isnull(),"cod_prov"] = 0
bank_data5.loc[bank_data5.cod_prov.isnull(),"cod_prov"] = 0
bank_data6.loc[bank_data6.cod_prov.isnull(),"cod_prov"] = 0
bank_data7.loc[bank_data7.cod_prov.isnull(),"cod_prov"] = 0
bank_data8.loc[bank_data8.cod_prov.isnull(),"cod_prov"] = 0

bank_data2.loc[bank_data2.cod_prov.isnull(),"cod_prov"] = 0
bank_data4.loc[bank_data4.cod_prov.isnull(),"cod_prov"] = 0
# bank_data5.loc[bank_data5.cod_prov.isnull(),"cod_prov"] = 0
bank_data3.loc[bank_data3.cod_prov.isnull(),"cod_prov"] = 0
# bank_data6.loc[bank_data6.cod_prov.isnull(),"cod_prov"] = 0
df_test.loc[df_test.cod_prov.isnull(),"cod_prov"] = 0

# bank_data7.loc[bank_data7.cod_prov.isnull(),"cod_prov"] = 0
# bank_data8.loc[bank_data8.cod_prov.isnull(),"cod_prov"] = 0
# bank_data9.loc[bank_data9.cod_prov.isnull(),"cod_prov"] = 0

Replacing the nan value in renta feature with its mean after grouping it with respect to cod_prov

In [ ]:
# bank_data0['renta'] = bank_data0.groupby('cod_prov')['renta'].apply(lambda x : x.fillna(x.mean()))
# bank_data0.loc[bank_data0.renta.isnull(), "renta"] = bank_data0['renta'].mean()
bank_data1['renta'] = bank_data1.groupby('cod_prov')['renta'].apply(lambda x : x.fillna(x.mean()))
bank_data1.loc[bank_data1.renta.isnull(), "renta"] = bank_data1['renta'].mean()
bank_data2['renta'] = bank_data2.groupby('cod_prov')['renta'].apply(lambda x : x.fillna(x.mean()))
bank_data2.loc[bank_data2.renta.isnull(), "renta"] = bank_data2['renta'].mean()
bank_data3['renta'] = bank_data3.groupby('cod_prov')['renta'].apply(lambda x : x.fillna(x.mean()))
bank_data3.loc[bank_data3.renta.isnull(), "renta"] = bank_data3['renta'].mean()
bank_data4['renta'] = bank_data4.groupby('cod_prov')['renta'].apply(lambda x : x.fillna(x.mean()))
bank_data4.loc[bank_data4.renta.isnull(), "renta"] = bank_data4['renta'].mean()
bank_data5['renta'] = bank_data5.groupby('cod_prov')['renta'].apply(lambda x : x.fillna(x.mean()))
bank_data5.loc[bank_data5.renta.isnull(), "renta"] = bank_data5['renta'].mean()
bank_data6['renta'] = bank_data6.groupby('cod_prov')['renta'].apply(lambda x : x.fillna(x.mean()))
bank_data6.loc[bank_data6.renta.isnull(), "renta"] = bank_data6['renta'].mean()
bank_data7['renta'] = bank_data7.groupby('cod_prov')['renta'].apply(lambda x : x.fillna(x.mean()))
bank_data7.loc[bank_data7.renta.isnull(), "renta"] = bank_data7['renta'].mean()
bank_data8['renta'] = bank_data8.groupby('cod_prov')['renta'].apply(lambda x : x.fillna(x.mean()))
bank_data8.loc[bank_data8.renta.isnull(), "renta"] = bank_data8['renta'].mean()
# bank_data9['renta'] = bank_data9.groupby('cod_prov')['renta'].apply(lambda x : x.fillna(x.mean()))
# bank_data9.loc[bank_data9.renta.isnull(), "renta"] = bank_data9['renta'].mean()


df_test['renta'] = df_test.groupby('cod_prov')['renta'].apply(lambda x : x.fillna(x.mean()))
df_test.loc[df_test.renta.isnull(), "renta"] = df_test['renta'].mean()

Sorting the dataframes based on ncodpers

In [ ]:
# bank_data0 = bank_data0.sort_values('ncodpers')
bank_data1 = bank_data1.sort_values('ncodpers')
bank_data2 = bank_data2.sort_values('ncodpers')
bank_data3 = bank_data3.sort_values('ncodpers')
bank_data4 = bank_data4.sort_values('ncodpers')
bank_data5 = bank_data5.sort_values('ncodpers')
bank_data6 = bank_data6.sort_values('ncodpers')
bank_data7 = bank_data7.sort_values('ncodpers')
bank_data8 = bank_data8.sort_values('ncodpers')
# bank_data9 = bank_data9.sort_values('ncodpers')
df_test = df_test.sort_values(by=['ncodpers'])

Adding a new column named 'indx' to keep track of index after sorting of df_test

In [ ]:
df_test['indx']=df_test.index

The product columns 'ind_nomina_ult1' and 'ind_nom_pens_ult1' have null values. They have been replaced by 0. Meaning that the products are not there and hence represented as 0.

In [ ]:
bank_data1.loc[bank_data1.ind_nomina_ult1.isnull(),"ind_nomina_ult1"] =  0
bank_data1.loc[bank_data1.ind_nom_pens_ult1.isnull(),"ind_nom_pens_ult1"] = 0
bank_data2.loc[bank_data2.ind_nomina_ult1.isnull(),"ind_nomina_ult1"] =  0
bank_data2.loc[bank_data2.ind_nom_pens_ult1.isnull(),"ind_nom_pens_ult1"] = 0
bank_data5.loc[bank_data5.ind_nomina_ult1.isnull(),"ind_nomina_ult1"] =  0
bank_data5.loc[bank_data5.ind_nom_pens_ult1.isnull(),"ind_nom_pens_ult1"] = 0
# bank_data0.loc[bank_data0.ind_nomina_ult1.isnull(),"ind_nomina_ult1"] =  0
# bank_data0.loc[bank_data0.ind_nom_pens_ult1.isnull(),"ind_nom_pens_ult1"] = 0
bank_data4.loc[bank_data4.ind_nomina_ult1.isnull(),"ind_nomina_ult1"] =  0
bank_data4.loc[bank_data4.ind_nom_pens_ult1.isnull(),"ind_nom_pens_ult1"] = 0

bank_data3.loc[bank_data3.ind_nomina_ult1.isnull(),"ind_nomina_ult1"] =  0
bank_data3.loc[bank_data3.ind_nom_pens_ult1.isnull(),"ind_nom_pens_ult1"] = 0

bank_data6.loc[bank_data6.ind_nomina_ult1.isnull(),"ind_nomina_ult1"] =  0
bank_data6.loc[bank_data6.ind_nom_pens_ult1.isnull(),"ind_nom_pens_ult1"] = 0
bank_data7.loc[bank_data7.ind_nomina_ult1.isnull(),"ind_nomina_ult1"] =  0
bank_data7.loc[bank_data7.ind_nom_pens_ult1.isnull(),"ind_nom_pens_ult1"] = 0
bank_data8.loc[bank_data8.ind_nomina_ult1.isnull(),"ind_nomina_ult1"] =  0
bank_data8.loc[bank_data8.ind_nom_pens_ult1.isnull(),"ind_nom_pens_ult1"] = 0
# bank_data9.loc[bank_data9.ind_nomina_ult1.isnull(),"ind_nomina_ult1"] =  0
# bank_data9.loc[bank_data9.ind_nom_pens_ult1.isnull(),"ind_nom_pens_ult1"] = 0

# Merging of dataframes

Adding 3 lags to april 2016 and may 2015 in train data. 

In [ ]:
nbank_data0 = bank_data1.copy()
bank_data4 = pd.merge(left = nbank_data0[['ncodpers','ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']],right = bank_data4, how ='right', on = ['ncodpers'],suffixes=('_lag3',''))

In [ ]:
bank_data5 = pd.merge(left = bank_data8[['ncodpers','ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']],right = bank_data5, how ='right', on = ['ncodpers'],suffixes=('_lag3',''))

In [ ]:
del(nbank_data0)

In [ ]:
nbank_data1 = bank_data2.copy()
bank_data4 = pd.merge(left = nbank_data1[['ncodpers','ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']],right = bank_data4, how ='right', on = ['ncodpers'],suffixes=('_lag2',''))

In [ ]:
bank_data5 = pd.merge(left = bank_data7[['ncodpers','ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']],right = bank_data5, how ='right', on = ['ncodpers'],suffixes=('_lag2',''))

In [ ]:
del(nbank_data1)

In [ ]:
nbank_data2 = bank_data3.copy()
bank_data4 = pd.merge(left = nbank_data2[['ncodpers','ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']],right = bank_data4, how ='right', on = ['ncodpers'],suffixes=('_lag1',''))

In [ ]:
bank_data5 = pd.merge(left = bank_data6[['ncodpers','ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']],right = bank_data5, how ='right', on = ['ncodpers'],suffixes=('_lag1',''))

In [ ]:
del(nbank_data2)

In [ ]:
bank_data = pd.DataFrame()
bank_data = pd.concat([bank_data,bank_data4])
bank_data = pd.concat([bank_data,bank_data5])

In [ ]:
bank_data

Adding 3 lags to test data

In [ ]:
nbank_data4 = bank_data4.copy()
df_test = pd.merge(left = nbank_data4[['ncodpers','ind_ahor_fin_ult1_lag2','ind_aval_fin_ult1_lag2','ind_cco_fin_ult1_lag2','ind_cder_fin_ult1_lag2','ind_cno_fin_ult1_lag2','ind_ctju_fin_ult1_lag2',
 'ind_ctma_fin_ult1_lag2','ind_ctop_fin_ult1_lag2','ind_ctpp_fin_ult1_lag2','ind_deco_fin_ult1_lag2','ind_deme_fin_ult1_lag2',
 'ind_dela_fin_ult1_lag2','ind_ecue_fin_ult1_lag2','ind_fond_fin_ult1_lag2','ind_hip_fin_ult1_lag2','ind_plan_fin_ult1_lag2',
 'ind_pres_fin_ult1_lag2','ind_reca_fin_ult1_lag2','ind_tjcr_fin_ult1_lag2','ind_valo_fin_ult1_lag2','ind_viv_fin_ult1_lag2','ind_nomina_ult1_lag2',
 'ind_nom_pens_ult1_lag2','ind_recibo_ult1_lag2']],right = df_test, how ='right', on = ['ncodpers'])

In [ ]:
df_test = df_test.rename(columns={ 'ind_ahor_fin_ult1_lag2':'ind_ahor_fin_ult1_lag3', 'ind_aval_fin_ult1_lag2':'ind_aval_fin_ult1_lag3',
       'ind_cco_fin_ult1_lag2' : 'ind_cco_fin_ult1_lag3', 'ind_cder_fin_ult1_lag2' : 'ind_cder_fin_ult1_lag3','ind_cno_fin_ult1_lag2': 'ind_cno_fin_ult1_lag3',
       'ind_ctju_fin_ult1_lag2' :'ind_ctju_fin_ult1_lag3','ind_ctma_fin_ult1_lag2' : 'ind_ctma_fin_ult1_lag3','ind_ctop_fin_ult1_lag2' : 'ind_ctop_fin_ult1_lag3',
       'ind_ctpp_fin_ult1_lag2':'ind_ctpp_fin_ult1_lag3', 'ind_deco_fin_ult1_lag2':'ind_deco_fin_ult1_lag3','ind_deme_fin_ult1_lag2': 'ind_deme_fin_ult1_lag3',
       'ind_dela_fin_ult1_lag2' : 'ind_dela_fin_ult1_lag3','ind_ecue_fin_ult1_lag2' : 'ind_ecue_fin_ult1_lag3','ind_fond_fin_ult1_lag2' :'ind_fond_fin_ult1_lag3',
       'ind_hip_fin_ult1_lag2' : 'ind_hip_fin_ult1_lag3', 'ind_plan_fin_ult1_lag2' :'ind_plan_fin_ult1_lag3','ind_pres_fin_ult1_lag2' : 'ind_pres_fin_ult1_lag3',
       'ind_reca_fin_ult1_lag2':'ind_reca_fin_ult1_lag3', 'ind_tjcr_fin_ult1_lag2':'ind_tjcr_fin_ult1_lag3', 'ind_valo_fin_ult1_lag2':'ind_valo_fin_ult1_lag3',
       'ind_viv_fin_ult1_lag2':'ind_viv_fin_ult1_lag3','ind_nomina_ult1_lag2' : 'ind_nomina_ult1_lag3','ind_nom_pens_ult1_lag2' : 'ind_nom_pens_ult1_lag3',
       'ind_recibo_ult1_lag2':'ind_recibo_ult1_lag3' })

In [ ]:
nbank_data4 = bank_data4.copy()
df_test = pd.merge(left = nbank_data4[['ncodpers','ind_ahor_fin_ult1_lag1','ind_aval_fin_ult1_lag1','ind_cco_fin_ult1_lag1','ind_cder_fin_ult1_lag1','ind_cno_fin_ult1_lag1','ind_ctju_fin_ult1_lag1',
 'ind_ctma_fin_ult1_lag1','ind_ctop_fin_ult1_lag1','ind_ctpp_fin_ult1_lag1','ind_deco_fin_ult1_lag1','ind_deme_fin_ult1_lag1',
 'ind_dela_fin_ult1_lag1','ind_ecue_fin_ult1_lag1','ind_fond_fin_ult1_lag1','ind_hip_fin_ult1_lag1','ind_plan_fin_ult1_lag1',
 'ind_pres_fin_ult1_lag1','ind_reca_fin_ult1_lag1','ind_tjcr_fin_ult1_lag1','ind_valo_fin_ult1_lag1','ind_viv_fin_ult1_lag1','ind_nomina_ult1_lag1',
 'ind_nom_pens_ult1_lag1','ind_recibo_ult1_lag1']],right = df_test, how ='right', on = ['ncodpers'])

In [ ]:
df_test = df_test.rename(columns={ 'ind_ahor_fin_ult1_lag1':'ind_ahor_fin_ult1_lag2', 'ind_aval_fin_ult1_lag1':'ind_aval_fin_ult1_lag2',
       'ind_cco_fin_ult1_lag1' : 'ind_cco_fin_ult1_lag2', 'ind_cder_fin_ult1_lag1' : 'ind_cder_fin_ult1_lag2','ind_cno_fin_ult1_lag1': 'ind_cno_fin_ult1_lag2',
       'ind_ctju_fin_ult1_lag1' :'ind_ctju_fin_ult1_lag2','ind_ctma_fin_ult1_lag1' : 'ind_ctma_fin_ult1_lag2','ind_ctop_fin_ult1_lag1' : 'ind_ctop_fin_ult1_lag2',
       'ind_ctpp_fin_ult1_lag1':'ind_ctpp_fin_ult1_lag2', 'ind_deco_fin_ult1_lag1':'ind_deco_fin_ult1_lag2','ind_deme_fin_ult1_lag1': 'ind_deme_fin_ult1_lag2',
       'ind_dela_fin_ult1_lag1' : 'ind_dela_fin_ult1_lag2','ind_ecue_fin_ult1_lag1' : 'ind_ecue_fin_ult1_lag2','ind_fond_fin_ult1_lag1' :'ind_fond_fin_ult1_lag2',
       'ind_hip_fin_ult1_lag1' : 'ind_hip_fin_ult1_lag2', 'ind_plan_fin_ult1_lag1' :'ind_plan_fin_ult1_lag2','ind_pres_fin_ult1_lag1' : 'ind_pres_fin_ult1_lag2',
       'ind_reca_fin_ult1_lag1':'ind_reca_fin_ult1_lag2', 'ind_tjcr_fin_ult1_lag1':'ind_tjcr_fin_ult1_lag2', 'ind_valo_fin_ult1_lag1':'ind_valo_fin_ult1_lag2',
       'ind_viv_fin_ult1_lag1':'ind_viv_fin_ult1_lag2','ind_nomina_ult1_lag1' : 'ind_nomina_ult1_lag2','ind_nom_pens_ult1_lag1' : 'ind_nom_pens_ult1_lag2',
       'ind_recibo_ult1_lag1':'ind_recibo_ult1_lag2' })

In [ ]:
df_test

In [ ]:
nbank_data4 = bank_data4.copy()
df_test = pd.merge(left = nbank_data4[['ncodpers','ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']],right = df_test, how ='right', on = ['ncodpers'])

In [ ]:
df_test = df_test.rename(columns={ 'ind_ahor_fin_ult1':'ind_ahor_fin_ult1_lag1', 'ind_aval_fin_ult1':'ind_aval_fin_ult1_lag1',
       'ind_cco_fin_ult1' : 'ind_cco_fin_ult1_lag1', 'ind_cder_fin_ult1' : 'ind_cder_fin_ult1_lag1','ind_cno_fin_ult1': 'ind_cno_fin_ult1_lag1',
       'ind_ctju_fin_ult1' :'ind_ctju_fin_ult1_lag1','ind_ctma_fin_ult1' : 'ind_ctma_fin_ult1_lag1','ind_ctop_fin_ult1' : 'ind_ctop_fin_ult1_lag1',
       'ind_ctpp_fin_ult1':'ind_ctpp_fin_ult1_lag1', 'ind_deco_fin_ult1':'ind_deco_fin_ult1_lag1','ind_deme_fin_ult1': 'ind_deme_fin_ult1_lag1',
       'ind_dela_fin_ult1' : 'ind_dela_fin_ult1_lag1','ind_ecue_fin_ult1' : 'ind_ecue_fin_ult1_lag1','ind_fond_fin_ult1' :'ind_fond_fin_ult1_lag1',
       'ind_hip_fin_ult1' : 'ind_hip_fin_ult1_lag1', 'ind_plan_fin_ult1' :'ind_plan_fin_ult1_lag1','ind_pres_fin_ult1' : 'ind_pres_fin_ult1_lag1',
       'ind_reca_fin_ult1':'ind_reca_fin_ult1_lag1', 'ind_tjcr_fin_ult1':'ind_tjcr_fin_ult1_lag1', 'ind_valo_fin_ult1':'ind_valo_fin_ult1_lag1',
       'ind_viv_fin_ult1':'ind_viv_fin_ult1_lag1','ind_nomina_ult1' : 'ind_nomina_ult1_lag1','ind_nom_pens_ult1' : 'ind_nom_pens_ult1_lag1',
       'ind_recibo_ult1':'ind_recibo_ult1_lag1' })

In [ ]:
df_test = df_test.sort_values(by=['indx'])
del df_test['indx']
del(nbank_data4)

Converting the fecha_dato and fecha_alta features to date-time frame

In [ ]:
bank_data["fecha_dato"] = pd.to_datetime(bank_data["fecha_dato"],format="%Y-%m-%d")
df_test["fecha_dato"] = pd.to_datetime(df_test["fecha_dato"],format="%Y-%m-%d")
bank_data["fecha_alta"] = pd.to_datetime(bank_data["fecha_alta"],format="%Y-%m-%d")
df_test["fecha_alta"] = pd.to_datetime(df_test["fecha_alta"],format="%Y-%m-%d")
bank_data["month"] = pd.DatetimeIndex(bank_data["fecha_dato"]).month
df_test["month"] = pd.DatetimeIndex(df_test["fecha_dato"]).month
bank_data["year"] = pd.DatetimeIndex(bank_data["fecha_dato"]).month
df_test["year"] = pd.DatetimeIndex(df_test["fecha_dato"]).month

In [ ]:
bank_data.month.unique()

# FILLING OF NAN VALUES

ind_empleado

In [ ]:
bank_data.loc[bank_data.ind_empleado.isnull(), "ind_empleado"] = "unknown"
bank_data['ind_empleado'].unique()    

pais_residencia

In [ ]:
bank_data.loc[bank_data.pais_residencia.isnull(), "pais_residencia"] = "unknown"
bank_data['pais_residencia'].unique()    

sexo

In [ ]:
bank_data.loc[bank_data.sexo.isnull(), "sexo"] = "unknown"
bank_data['sexo'].unique()    

indrel

In [ ]:
bank_data.loc[bank_data.indrel.isnull(),"indrel"] = bank_data['indrel'].median()
bank_data['indrel'].unique()

indrel_1mes

In [ ]:
bank_data.indrel_1mes.mode()[0]
bank_data.loc[bank_data.indrel_1mes.isnull(), "indrel_1mes"] = 1.0
bank_data.indrel_1mes.unique()

tiprel_1mes

In [ ]:
bank_data.loc[bank_data.tiprel_1mes.isnull(), "tiprel_1mes"] = "unknown"
bank_data['tiprel_1mes'].unique()

indext

In [ ]:
bank_data.loc[bank_data.indext.isnull(), "indext"] = "unknown"
bank_data['indext'].unique()

conyuemp

In [ ]:
bank_data.loc[bank_data.conyuemp.isnull(), "conyuemp"] = "unknown"
bank_data['conyuemp'].unique()

canal_entrada

In [ ]:
bank_data.loc[bank_data.canal_entrada.isnull(), "canal_entrada"] = "unknown"
bank_data['canal_entrada'].unique()

indfall

In [ ]:
bank_data.loc[bank_data.indfall.isnull(), "indfall"] = "unknown"
bank_data['indfall'].unique()

cod_prov

In [ ]:
bank_data.loc[bank_data.cod_prov.isnull(),"cod_prov"] = 0
bank_data['cod_prov'].unique()

In [ ]:
bank_data.loc[bank_data.antiguedad.isnull(),"cod_prov"] = 0

ind_actividad_clienta

In [ ]:
bank_data.loc[bank_data.ind_actividad_cliente.isnull(),"ind_actividad_cliente"] = bank_data['ind_actividad_cliente'].median()
bank_data['ind_actividad_cliente'].unique()

segmento

In [ ]:
bank_data.loc[bank_data.segmento.isnull(), "segmento"] = "04-unknown"
bank_data['segmento'].unique()

In [ ]:
lag_col=['ind_ahor_fin_ult1_lag','ind_aval_fin_ult1_lag','ind_cco_fin_ult1_lag','ind_cder_fin_ult1_lag','ind_cno_fin_ult1_lag','ind_ctju_fin_ult1_lag',
         'ind_ctma_fin_ult1_lag','ind_ctop_fin_ult1_lag','ind_ctpp_fin_ult1_lag','ind_deco_fin_ult1_lag','ind_deme_fin_ult1_lag','ind_dela_fin_ult1_lag',
         'ind_ecue_fin_ult1_lag','ind_fond_fin_ult1_lag','ind_hip_fin_ult1_lag','ind_plan_fin_ult1_lag','ind_pres_fin_ult1_lag','ind_reca_fin_ult1_lag',
         'ind_tjcr_fin_ult1_lag','ind_valo_fin_ult1_lag','ind_viv_fin_ult1_lag','ind_nomina_ult1_lag','ind_nom_pens_ult1_lag','ind_recibo_ult1_lag']

In [ ]:
bank_data.loc[bank_data.ult_fec_cli_1t.isnull(), "ult_fec_cli_1t"] = "unknown"
bank_data.loc[bank_data.indresi.isnull(), "indresi"] = "unknown"
bank_data.loc[bank_data.nomprov.isnull(), "nomprov"] = "unknown"
bank_data.loc[bank_data.tipodom.isnull(),"tipodom"] = bank_data['tipodom'].median()
bank_data.loc[bank_data.ind_nuevo.isnull(),"ind_nuevo"] = bank_data['ind_nuevo'].median()

#filling nan values in lag_columns with '0'
for j in range(1,4):
    for i in lag_col:
        lag_name=i+str(j)
        bank_data.loc[bank_data[lag_name].isnull(),lag_name] = 0
        df_test.loc[df_test[lag_name].isnull(),lag_name] = 0

# FILLLING OF NAN VALUES IN TEST DATA

In [ ]:
df_test.info()

In [ ]:
props = df_test.fillna(np.nan)
df_test.isnull().sum()

sexo

In [ ]:
df_test.loc[df_test.sexo.isnull(), "sexo"] = "unknown"
df_test['sexo'].unique()

ult_fec_cli_1t

In [ ]:
df_test.loc[df_test.ult_fec_cli_1t.isnull(), "ult_fec_cli_1t"] = "unknown"
df_test['ult_fec_cli_1t'].unique()

indrel_1mes

In [ ]:
df_test.loc[df_test.indrel_1mes.isnull(), "indrel_1mes"] = "unknown"
df_test['indrel_1mes'].unique()

tiprel_1mes

In [ ]:
df_test.loc[df_test.tiprel_1mes.isnull(), "tiprel_1mes"] = "unknown"
df_test['tiprel_1mes'].unique()

conyuemp

In [ ]:
df_test.loc[df_test.conyuemp.isnull(), "conyuemp"] = "unknown"
df_test['conyuemp'].unique()

canal_entrada

In [ ]:
df_test.loc[df_test.canal_entrada.isnull(), "canal_entrada"] = "unknown"
df_test['canal_entrada'].unique()

cod_prov

In [ ]:
df_test.loc[df_test.cod_prov.isnull(),"cod_prov"] = 0
df_test['cod_prov'].unique()

nomprov

In [ ]:
df_test.loc[df_test.nomprov.isnull(), "nomprov"] = "unknown"
df_test['nomprov'].unique()

segmento

In [ ]:
df_test.loc[df_test.segmento.isnull(), "segmento"] = "04-unknown"
df_test['segmento'].unique()

In [ ]:
props = df_test.fillna(np.nan)
df_test.isnull().sum()

In [ ]:
props = bank_data.fillna(np.nan)
bank_data.isnull().sum()

In [ ]:
df_recent = bank_data[['ncodpers'] + product_col]

# MERGING OF PRODUCTS COLUMNS IN TRAIN DATA

In [ ]:
bank_data.size

In [ ]:
product_list={0:'ind_ahor_fin_ult1',1:'ind_aval_fin_ult1',2:'ind_cco_fin_ult1',3:'ind_cder_fin_ult1',4:'ind_cno_fin_ult1',5:'ind_ctju_fin_ult1',6:'ind_ctma_fin_ult1',7:'ind_ctop_fin_ult1',8: 'ind_ctpp_fin_ult1',
       9:'ind_deco_fin_ult1', 10:'ind_deme_fin_ult1', 11:'ind_dela_fin_ult1',12:'ind_ecue_fin_ult1', 13:'ind_fond_fin_ult1',14: 'ind_hip_fin_ult1',15:'ind_plan_fin_ult1',16: 'ind_pres_fin_ult1',17: 'ind_reca_fin_ult1',
       18:'ind_tjcr_fin_ult1', 19:'ind_valo_fin_ult1', 20:'ind_viv_fin_ult1',21:'ind_nomina_ult1',22: 'ind_nom_pens_ult1',23: 'ind_recibo_ult1'}
bank_data['new']=""
for i in range(24):
  bank_data['new']=bank_data['new']+(bank_data[product_list[i]].astype(int).astype(str))
  del bank_data[product_list[i]]
bank_data.size

The size has been reduced significantly

# DROPPING OF COLUMNS

nom_prov

In [ ]:
bank_data = bank_data.sort_values('cod_prov')

In [ ]:
sns.scatterplot(data=bank_data,x="nomprov",y="cod_prov")

In [ ]:
bank_data.drop('nomprov', axis=1, inplace=True)
df_test.drop('nomprov', axis=1, inplace=True)

ult_fec_cli_1t

In [ ]:
sns.scatterplot(data=bank_data,y="indrel",x="ult_fec_cli_1t")

In [ ]:
bank_data.drop('ult_fec_cli_1t', axis=1, inplace=True)
df_test.drop('ult_fec_cli_1t', axis=1, inplace=True)

indresi

In [ ]:
sns.scatterplot(data=bank_data,y="indresi",x="pais_residencia")

In [ ]:
bank_data.drop('indresi', axis=1, inplace=True)
df_test.drop('indresi', axis=1, inplace=True)

antiguedad

In [ ]:
bank_data['antiguedad']=bank_data['antiguedad'].astype(str)

In [ ]:
bank_data.loc[bank_data.fecha_alta.isnull(), "fecha_alta"] =bank_data['fecha_alta'].mode()

In [ ]:
bank_data

In [ ]:
sns.scatterplot(data=bank_data,x="antiguedad",y="fecha_alta")

In [ ]:
bank_data.drop(('antiguedad'), axis=1, inplace=True)  
df_test.drop(('antiguedad'), axis=1, inplace=True)   

ind_nuevo

In [ ]:
bank_data.drop(('ind_nuevo'), axis=1, inplace=True)  
df_test.drop(('ind_nuevo'), axis=1, inplace=True)  

tipodom

In [ ]:
sns.scatterplot(data=bank_data,x="tipodom",y="pais_residencia")

In [ ]:
bank_data.drop(('tipodom'), axis=1, inplace=True) 
df_test.drop(('tipodom'), axis=1, inplace=True) 

fecha_alta and fecha_dato

In [ ]:
bank_data.drop(("fecha_dato"), axis=1, inplace=True) 
df_test.drop(("fecha_dato"), axis=1, inplace=True) 
bank_data.drop(('fecha_alta'), axis=1, inplace=True) 
df_test.drop(('fecha_alta'), axis=1, inplace=True) 


Age

In [ ]:
bank_data["age"]   = pd.to_numeric(bank_data["age"], errors="coerce")
df_test["age"]   = pd.to_numeric(df_test["age"], errors="coerce")

age_group = [[0,18],[18,28],[28,38],[38,48],[48,58],[58,68],[68,78],[78,88],[88,170]]

def create_age_group(df):  
    df['age_group'] = np.nan
    for i,age in enumerate(age_group):
        row_index = (df['age'] >= age[0]) & (df['age'] < age[1])
        df.loc[row_index,'age_group'] = i
        
create_age_group(bank_data)
create_age_group(df_test)

bank_data.drop('age', axis=1, inplace = True)
df_test.drop('age', axis=1, inplace = True)

In [ ]:
bank_data["age_group"].fillna(bank_data["age_group"].mode()[0].astype(float),inplace = True)

renta

In [ ]:
renta_group = [[0,50000],[50000,70000],[70000,100000],[100000,150000],[150000,200000],[200000,1000000],[1000000,29000000]]

def create_renta_group(df):  
    df['renta_group'] = np.nan
    for i,renta in enumerate(renta_group):
        row_index = (df['renta'] >= renta[0]) & (df['renta'] < renta[1])
        df.loc[row_index,'renta_group'] = i
        
create_renta_group(bank_data)
create_renta_group(df_test)

bank_data.drop('renta', axis=1, inplace = True)
df_test.drop('renta', axis=1, inplace = True)

df_test.renta_group.unique()

In [ ]:
props = bank_data.fillna(np.nan)
bank_data.isnull().sum()

In [ ]:
props = df_test.fillna(np.nan)
df_test.isnull().sum()

To keep track of products a customer has, we have already_active dictionary

In [ ]:
already_active = {}

for row in df_recent.values:
    row = list(row)
    id = row.pop(0)
    s=zip(tuple(product_col), row)
    active = [c[0] for c in s if int(c[1]) > 0]
    already_active[id] = active    
    zip(*s)

# ENCODING

Selecting only object datatypes columns to a data frame for encoding

In [ ]:
df_cat= bank_data.select_dtypes(include = 'object').astype(str).copy()
test_cat= df_test.select_dtypes(include = 'object').astype(str).copy()

In [ ]:
df_cat.drop(("new"), axis=1, inplace=True)
df_cat

In [ ]:
for i in df_cat:
    del bank_data[i]
for j in test_cat:
    del df_test[j]

In [ ]:
onehot=OneHotEncoder(handle_unknown ='ignore')
df_hot=onehot.fit_transform(df_cat)
test_hot=onehot.transform(test_cat)

In [ ]:
bank_data['new'].copy()

In [ ]:
just_code = bank_data['new'].copy()

In [ ]:
df_new = pd.DataFrame(just_code) 

In [ ]:
df_new['new']

In [ ]:
bank_data.drop(columns = ['new','ncodpers'],inplace = True)

In [ ]:
ids = df_test['ncodpers'].values
sample=df_test['ncodpers']
df_test.drop(columns = ['ncodpers'],inplace = True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
norm = MinMaxScaler().fit(bank_data)
X_train_norm = norm.transform(bank_data)
X_test_norm = norm.transform(df_test)
del bank_data
del df_test
x_train=X_train_norm.copy()
x_test=X_test_norm.copy()
del X_train_norm
del X_test_norm

In [ ]:
bank_data=pd.DataFrame(data=x_train)

In [ ]:
df_test=pd.DataFrame(data=x_test)

In [ ]:
import scipy.sparse as sp
A=sp.csr_matrix(bank_data.values)
df=sp.csr_matrix.transpose(A)

In [ ]:
B=sp.csr_matrix(df_test.values)
df_1=sp.csr_matrix.transpose(B)
df_1=df_1.T
df_test=sp.hstack((df_1,test_hot))

In [ ]:
x_test = df_test.copy()

In [ ]:
df_train=sp.hstack((bank_data,df_hot))
x_train=df_train.copy()

# MODELLING

In [ ]:
id_preds = defaultdict(list)
c=0
for c in range(24):
        y_train = df_new['new'].astype(str).str[c].astype(int)
        clf = LogisticRegression()
        clf.fit(x_train, y_train)

        p_train = clf.predict_proba(x_test)[:,1]
        for id, p in zip(ids, p_train):
            id_preds[id].append(p)

In [ ]:
id_preds

In [ ]:
train_preds = {}
for id, p in id_preds.items():
    preds=[]
    ps=[]
    preds_p=[]
    ps_p=[]
    if id in already_active:
        preds=preds+([i[0] for i in sorted([i for i in zip(tuple(product_col), p )if i[0] not in already_active[id]],
                                  key=lambda i:i [1], 
                                  reverse=True)[:5]])
        preds_p=preds_p+([i[1] for i in sorted([i for i in zip(tuple(product_col), p) if i[0] not in already_active[id]],
                                  key=lambda i:i [1], 
                                  reverse=True)[:5]])
        ps=ps+([i[0] for i in sorted([i for i in zip(tuple(product_col), p) if i[0] in already_active[id]],
                                  key=lambda i:i [1])])
        ps_p=ps_p+([i[1] for i in sorted([i for i in zip(tuple(product_col), p) if i[0] in already_active[id]],
                                  key=lambda i:i [1])]) 
        if len(ps)>5:
            ps=ps[:5]
            ps_p=ps_p[:5]
        for i in ps_p:
            i=1-i
        preds=preds+ps
        preds_p=preds_p+ps_p
        preds=([i[0] for i in sorted(zip(tuple(preds),tuple(preds_p)), key=lambda i:i [1],reverse=True)])[:5]
      
    else:
        preds=preds+([i[0] for i in sorted([i for i in zip(tuple(product_col), p )],
                                  key=lambda i:i [1], 
                                  reverse=True)[:5]])    
    train_preds[id] = preds
    
train_preds

In [ ]:
test_preds = []
for row in ids:
    id = row
    p = train_preds[id]
    test_preds.append(' '.join(p))

In [ ]:
sample['added_products'] = test_preds

In [ ]:
sample

In [ ]:
submission = pd.DataFrame({'ncodpers': sample[0:931453],'changed': test_preds[0:931453] })

In [ ]:
submission

In [ ]:
submission.to_csv('LR_sub.csv', index=False)
print(datetime.datetime.now()-start_time)